In [1]:
%SparkSession

# SparkSession - Delta
| Spark Session        | http://localhost:4040        |
|----------------------|------------------------------|
| Spark Master         | http://localhost:5050        |
| Spark Worker A       | http://localhost:5051        |
| Spark Worker B       | http://localhost:5052        | 
| Hive Metastore       | thrift://hive-metastore:9083 |

| MinIO Object Storage | http://localhost:9090 |
|----------------------|-----------------------|
| user                 | `admin`               |
| password             | `password`            |

- Use `%trino` or `%%trino` to run queries using Trino
- Use `%sparksql` or `%%sparksql` to run queries using SparkSQL


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SparkSession-Delta") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.instances", "1") \
    .enableHiveSupport() \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a18371ab-429b-4d0d-8bcf-2d251bb3a040;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 132ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0  

In [2]:
df = spark.read.csv("s3a://kevin/combined_financial_data_idx.csv", header=True, inferSchema=True)

In [3]:
df.show()

+------+--------------------+----+-------------+-------------+-------------+-------------+
|symbol|             account|type|         2020|         2021|         2022|         2023|
+------+--------------------+----+-------------+-------------+-------------+-------------+
|  AALI|    Accounts Payable|  BS|   7.70264E11|  1.026717E12|  1.224423E12|   8.42064E11|
|  AALI| Accounts Receivable|  BS|   7.65849E11|   4.58135E11|    8.4877E11|   6.74487E11|
|  AALI|Accumulated Depre...|  BS|-1.0920948E13|-1.2133813E13|-1.3303749E13|-1.4436847E13|
|  AALI|Additional Paid I...|  BS|  3.878995E12|  3.878995E12|  3.878995E12|  3.878995E12|
|  AALI|Allowance For Dou...|  BS|   -2.4261E10|   -2.4543E10|   -2.7057E10|   -2.6516E10|
|  AALI|Basic Average Shares|  IS|1.924688333E9|1.924688333E9|1.924688333E9|1.924688333E9|
|  AALI|           Basic EPS|  IS|       432.84|      1024.25|       897.08|       548.61|
|  AALI|Beginning Cash Po...|  CF|   3.83366E11|   9.78892E11|  3.896022E12|  1.619616E12|

In [4]:
df.write.format("delta").mode("append").save("default.sample_delta_tbl")

Py4JJavaError: An error occurred while calling o73.save.
: java.lang.NoSuchMethodError: 'scala.collection.Seq org.apache.spark.sql.types.StructType.toAttributes()'
	at org.apache.spark.sql.delta.files.TransactionalWrite.getStatsSchema(TransactionalWrite.scala:262)
	at org.apache.spark.sql.delta.files.TransactionalWrite.getStatsSchema$(TransactionalWrite.scala:252)
	at org.apache.spark.sql.delta.OptimisticTransaction.getStatsSchema(OptimisticTransaction.scala:137)
	at org.apache.spark.sql.delta.files.TransactionalWrite.getOptionalStatsTrackerAndStatsCollection(TransactionalWrite.scala:294)
	at org.apache.spark.sql.delta.files.TransactionalWrite.getOptionalStatsTrackerAndStatsCollection$(TransactionalWrite.scala:286)
	at org.apache.spark.sql.delta.OptimisticTransaction.getOptionalStatsTrackerAndStatsCollection(OptimisticTransaction.scala:137)
	at org.apache.spark.sql.delta.files.TransactionalWrite.writeFiles(TransactionalWrite.scala:352)
	at org.apache.spark.sql.delta.files.Transactional

In [ ]:
spark.stop()

In [14]:
spark.sql("SHOW TABLES").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default| delta_table|      false|
|  default|      sample|      false|
|  default|test_table_3|      false|
+---------+------------+-----------+



In [16]:
spark.stop()